In [1]:
import polars as pl
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score 

dataPath = "/kaggle/input/home-credit-credit-risk-model-stability/"

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


In [2]:
def set_table_dtypes(df: pl.DataFrame) -> pl.DataFrame:
    # implement here all desired dtypes for tables
    # the following is just an example
    for col in df.columns:
        # last letter of column name will help you determine the type
        if col[-1] in ("P", "A"):
            df = df.with_columns(pl.col(col).cast(pl.Float64).alias(col))

    return df

def convert_strings(df: pd.DataFrame) -> pd.DataFrame:
    for col in df.columns:  
        if df[col].dtype.name in ['object', 'string']:
            df[col] = df[col].astype("string").astype('category')
            current_categories = df[col].cat.categories
            new_categories = current_categories.to_list() + ["Unknown"]
            new_dtype = pd.CategoricalDtype(categories=new_categories, ordered=True)
            df[col] = df[col].astype(new_dtype)
    return df

In [3]:
train_basetable = pl.read_csv(dataPath + "csv_files/train/train_base.csv")
train_static = pl.concat(
    [
        pl.read_csv(dataPath + "csv_files/train/train_static_0_0.csv").pipe(set_table_dtypes),
        pl.read_csv(dataPath + "csv_files/train/train_static_0_1.csv").pipe(set_table_dtypes),
    ],
    how="vertical_relaxed",
)
train_static_cb = pl.read_csv(dataPath + "csv_files/train/train_static_cb_0.csv").pipe(set_table_dtypes)
train_person_1 = pl.read_csv(dataPath + "csv_files/train/train_person_1.csv").pipe(set_table_dtypes) 
train_credit_bureau_b_2 = pl.read_csv(dataPath + "csv_files/train/train_credit_bureau_b_2.csv").pipe(set_table_dtypes) 

In [4]:
train_credit_bureau_b_2.head()

case_id,num_group1,num_group2,pmts_date_1107D,pmts_dpdvalue_108P,pmts_pmtsoverdue_635A
i64,i64,i64,str,f64,f64
467,2,19,"""2018-05-15""",0.0,0.0
467,2,25,"""2018-11-15""",0.0,0.0
467,2,18,"""2018-04-15""",0.0,0.0
467,2,0,"""2016-10-15""",0.0,0.0
467,2,6,"""2017-04-15""",0.0,0.0


In [5]:
test_basetable = pl.read_csv(dataPath + "csv_files/test/test_base.csv")
test_static = pl.concat(
    [
        pl.read_csv(dataPath + "csv_files/test/test_static_0_0.csv").pipe(set_table_dtypes),
        pl.read_csv(dataPath + "csv_files/test/test_static_0_1.csv").pipe(set_table_dtypes),
        pl.read_csv(dataPath + "csv_files/test/test_static_0_2.csv").pipe(set_table_dtypes),
    ],
    how="vertical_relaxed",
)
test_static_cb = pl.read_csv(dataPath + "csv_files/test/test_static_cb_0.csv").pipe(set_table_dtypes)
test_person_1 = pl.read_csv(dataPath + "csv_files/test/test_person_1.csv").pipe(set_table_dtypes) 
test_credit_bureau_b_2 = pl.read_csv(dataPath + "csv_files/test/test_credit_bureau_b_2.csv").pipe(set_table_dtypes) 

## Feature engineering

In this part, we can see a simple example of joining tables via `case_id`. Here the loading and joining is done with polars library. Polars library is blazingly fast and has much smaller memory footprint than pandas. 

In [6]:
# We need to use aggregation functions in tables with depth > 1, so tables that contain num_group1 column or 
# also num_group2 column.
train_person_1_feats_1 = train_person_1.group_by("case_id").agg(
    pl.col("mainoccupationinc_384A").max().alias("mainoccupationinc_384A_max"),
    (pl.col("incometype_1044T") == "SELFEMPLOYED").max().alias("mainoccupationinc_384A_any_selfemployed")
)

# Here num_group1=0 has special meaning, it is the person who applied for the loan.
train_person_1_feats_2 = train_person_1.select(["case_id", "num_group1", "housetype_905L"]).filter(
    pl.col("num_group1") == 0
).drop("num_group1").rename({"housetype_905L": "person_housetype"})

# Here we have num_goup1 and num_group2, so we need to aggregate again.
train_credit_bureau_b_2_feats = train_credit_bureau_b_2.group_by("case_id").agg(
    pl.col("pmts_pmtsoverdue_635A").max().alias("pmts_pmtsoverdue_635A_max"),
    (pl.col("pmts_dpdvalue_108P") > 31).max().alias("pmts_dpdvalue_108P_over31")
)

# We will process in this examples only A-type and M-type columns, so we need to select them.
selected_static_cols = []
for col in train_static.columns:
    if col[-1] in ("P","A", "M","D","T","L"):
        selected_static_cols.append(col)
print(selected_static_cols)

selected_static_cb_cols = []
for col in train_static_cb.columns:
    if col[-1] in ("P","A", "M","D","T","L"):
        selected_static_cb_cols.append(col)
print(selected_static_cb_cols)

# Join all tables together.
data = train_basetable.join(
    train_static.select(["case_id"]+selected_static_cols), how="left", on="case_id"
).join(
    train_static_cb.select(["case_id"]+selected_static_cb_cols), how="left", on="case_id"
).join(
    train_person_1_feats_1, how="left", on="case_id"
).join(
    train_person_1_feats_2, how="left", on="case_id"
).join(
    train_credit_bureau_b_2_feats, how="left", on="case_id"
)

['actualdpdtolerance_344P', 'amtinstpaidbefduel24m_4187115A', 'annuity_780A', 'annuitynextmonth_57A', 'applicationcnt_361L', 'applications30d_658L', 'applicationscnt_1086L', 'applicationscnt_464L', 'applicationscnt_629L', 'applicationscnt_867L', 'avgdbddpdlast24m_3658932P', 'avgdbddpdlast3m_4187120P', 'avgdbdtollast24m_4525197P', 'avgdpdtolclosure24_3658938P', 'avginstallast24m_3658937A', 'avglnamtstart24m_4525187A', 'avgmaxdpdlast9m_3716943P', 'avgoutstandbalancel6m_4187114A', 'avgpmtlast12m_4525200A', 'bankacctype_710L', 'cardtype_51L', 'clientscnt12m_3712952L', 'clientscnt3m_3712950L', 'clientscnt6m_3712949L', 'clientscnt_100L', 'clientscnt_1022L', 'clientscnt_1071L', 'clientscnt_1130L', 'clientscnt_136L', 'clientscnt_157L', 'clientscnt_257L', 'clientscnt_304L', 'clientscnt_360L', 'clientscnt_493L', 'clientscnt_533L', 'clientscnt_887L', 'clientscnt_946L', 'cntincpaycont9m_3716944L', 'cntpmts24_3658933L', 'commnoinclast6m_3546845L', 'credamount_770A', 'credtype_322L', 'currdebt_22A',

In [7]:
data.head()

case_id,date_decision,MONTH,WEEK_NUM,target,actualdpdtolerance_344P,amtinstpaidbefduel24m_4187115A,annuity_780A,annuitynextmonth_57A,applicationcnt_361L,applications30d_658L,applicationscnt_1086L,applicationscnt_464L,applicationscnt_629L,applicationscnt_867L,avgdbddpdlast24m_3658932P,avgdbddpdlast3m_4187120P,avgdbdtollast24m_4525197P,avgdpdtolclosure24_3658938P,avginstallast24m_3658937A,avglnamtstart24m_4525187A,avgmaxdpdlast9m_3716943P,avgoutstandbalancel6m_4187114A,avgpmtlast12m_4525200A,bankacctype_710L,cardtype_51L,clientscnt12m_3712952L,clientscnt3m_3712950L,clientscnt6m_3712949L,clientscnt_100L,clientscnt_1022L,clientscnt_1071L,clientscnt_1130L,clientscnt_136L,clientscnt_157L,clientscnt_257L,clientscnt_304L,…,formonth_206L,formonth_535L,forquarter_1017L,forquarter_462L,forquarter_634L,fortoday_1092L,forweek_1077L,forweek_528L,forweek_601L,foryear_618L,foryear_818L,foryear_850L,fourthquarter_440L,maritalst_385M,maritalst_893M,numberofqueries_373L,pmtaverage_3A,pmtaverage_4527227A,pmtaverage_4955615A,pmtcount_4527229L,pmtcount_4955617L,pmtcount_693L,pmtscount_423L,pmtssum_45A,requesttype_4525192L,responsedate_1012D,responsedate_4527233D,responsedate_4917613D,riskassesment_302T,riskassesment_940T,secondquarter_766L,thirdquarter_1082L,mainoccupationinc_384A_max,mainoccupationinc_384A_any_selfemployed,person_housetype,pmts_pmtsoverdue_635A_max,pmts_dpdvalue_108P_over31
i64,str,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,f64,f64,f64,f64,str,str,f64,f64,f64,str,str,str,str,str,f64,f64,f64,f64,bool,str,f64,bool
0,"""2019-01-03""",201901,0,0,null,null,1917.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,0.0,0.0,0.0,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,10800.0,false,null,null,null
1,"""2019-01-03""",201901,0,0,null,null,3134.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""0.0""",3.0,0.0,0.0,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,10000.0,false,null,null,null
2,"""2019-01-04""",201901,0,0,null,null,4937.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,0.0,0.0,0.0,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,14000.0,false,null,null,null
3,"""2019-01-03""",201901,0,0,null,null,4643.6,0.0,0.0,1.0,0.0,2.0,0.0,1.0,null,null,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,0.0,0.0,0.0,1.0,null,0.0,0.0,0.0,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,10000.0,false,null,null,null
4,"""2019-01-04""",201901,0,1,null,null,3390.2,0.0,0.0,1.0,0.0,0.0,0.0,1.0,null,null,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,0.0,0.0,0.0,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,24000.0,false,null,null,null


In [8]:
ct = data.select(pl.all().is_null().sum())

In [9]:
ct

case_id,date_decision,MONTH,WEEK_NUM,target,actualdpdtolerance_344P,amtinstpaidbefduel24m_4187115A,annuity_780A,annuitynextmonth_57A,applicationcnt_361L,applications30d_658L,applicationscnt_1086L,applicationscnt_464L,applicationscnt_629L,applicationscnt_867L,avgdbddpdlast24m_3658932P,avgdbddpdlast3m_4187120P,avgdbdtollast24m_4525197P,avgdpdtolclosure24_3658938P,avginstallast24m_3658937A,avglnamtstart24m_4525187A,avgmaxdpdlast9m_3716943P,avgoutstandbalancel6m_4187114A,avgpmtlast12m_4525200A,bankacctype_710L,cardtype_51L,clientscnt12m_3712952L,clientscnt3m_3712950L,clientscnt6m_3712949L,clientscnt_100L,clientscnt_1022L,clientscnt_1071L,clientscnt_1130L,clientscnt_136L,clientscnt_157L,clientscnt_257L,clientscnt_304L,…,formonth_206L,formonth_535L,forquarter_1017L,forquarter_462L,forquarter_634L,fortoday_1092L,forweek_1077L,forweek_528L,forweek_601L,foryear_618L,foryear_818L,foryear_850L,fourthquarter_440L,maritalst_385M,maritalst_893M,numberofqueries_373L,pmtaverage_3A,pmtaverage_4527227A,pmtaverage_4955615A,pmtcount_4527229L,pmtcount_4955617L,pmtcount_693L,pmtscount_423L,pmtssum_45A,requesttype_4525192L,responsedate_1012D,responsedate_4527233D,responsedate_4917613D,riskassesment_302T,riskassesment_940T,secondquarter_766L,thirdquarter_1082L,mainoccupationinc_384A_max,mainoccupationinc_384A_any_selfemployed,person_housetype,pmts_pmtsoverdue_635A_max,pmts_dpdvalue_108P_over31
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,…,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,418178,561124,0,4,0,0,0,0,0,0,613202,948244,972827,467175,624875,1364150,757006,841181,1026987,1109629,1334968,0,0,0,0,0,0,0,1526242,0,0,0,…,1490145,1490145,1490145,1490145,1490145,1490145,1490145,1490145,1490145,1490145,1490145,1490145,140968,26183,26183,140968,1383070,1411681,1454814,1411681,1454814,1380253,954021,954021,853395,806659,866332,1301747,1473100,1473099,140968,140968,0,0,1425841,1490244,1490244


In [10]:
total = int(data.select(pl.count()).item())
total

1526659

In [11]:
cols = ct.columns
new_df = data
# cols

In [12]:

for c in ct.rows():
    for i,j in zip(c,cols):
        null_t = 30
        null_perct = ((int(i)/total)*100)
#         print(null_perct)
        if(null_perct >= null_t):
#             print(j)
            new_df = new_df.drop(j)


In [13]:
new_df.head()

case_id,date_decision,MONTH,WEEK_NUM,target,actualdpdtolerance_344P,annuity_780A,annuitynextmonth_57A,applicationcnt_361L,applications30d_658L,applicationscnt_1086L,applicationscnt_464L,applicationscnt_629L,applicationscnt_867L,clientscnt12m_3712952L,clientscnt3m_3712950L,clientscnt6m_3712949L,clientscnt_100L,clientscnt_1022L,clientscnt_1071L,clientscnt_1130L,clientscnt_157L,clientscnt_257L,clientscnt_304L,clientscnt_360L,clientscnt_493L,clientscnt_533L,clientscnt_887L,clientscnt_946L,cntincpaycont9m_3716944L,commnoinclast6m_3546845L,credamount_770A,credtype_322L,currdebt_22A,currdebtcredtyperange_828A,daysoverduetolerancedd_3976961L,deferredmnthsnum_166L,…,numinstunpaidmax_3546851L,numnotactivated_1143L,numpmtchanneldd_318L,numrejects9m_859L,opencred_647L,paytype1st_925L,paytype_783L,pmtnum_254L,posfpd10lastmonth_333P,posfpd30lastmonth_3976960P,posfstqpd30lastmonth_3976962P,previouscontdistrict_112M,price_1097A,sellerplacecnt_915L,sellerplacescnt_216L,sumoutstandtotal_3546847A,totaldebt_9A,totalsettled_863A,twobodfilling_608L,dateofbirth_337D,days120_123L,days180_256L,days30_165L,days360_512L,days90_310L,description_5085714M,education_1103M,education_88M,firstquarter_103L,fourthquarter_440L,maritalst_385M,maritalst_893M,numberofqueries_373L,secondquarter_766L,thirdquarter_1082L,mainoccupationinc_384A_max,mainoccupationinc_384A_any_selfemployed
i64,str,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,…,f64,f64,f64,f64,bool,str,str,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,str,str,f64,f64,f64,f64,f64,str,str,str,f64,f64,str,str,f64,f64,f64,f64,bool
0,"""2019-01-03""",201901,0,0,null,1917.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,0.0,30000.0,"""CAL""",0.0,0.0,null,0.0,…,null,0.0,0.0,0.0,null,"""OTHER""","""OTHER""",24.0,0.0,0.0,null,"""a55475b1""",null,0.0,0.0,null,0.0,0.0,"""BO""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,10800.0,false
1,"""2019-01-03""",201901,0,0,null,3134.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,0.0,19999.8,"""CAL""",0.0,0.0,null,0.0,…,null,0.0,0.0,0.0,null,"""OTHER""","""OTHER""",18.0,0.0,0.0,null,"""a55475b1""",null,0.0,0.0,null,0.0,0.0,"""BO""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,10000.0,false
2,"""2019-01-04""",201901,0,0,null,4937.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,78000.0,"""CAL""",0.0,0.0,null,0.0,…,null,0.0,0.0,0.0,false,"""OTHER""","""OTHER""",36.0,0.0,0.0,null,"""a55475b1""",null,0.0,0.0,null,0.0,0.0,"""BO""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,14000.0,false
3,"""2019-01-03""",201901,0,0,null,4643.6,0.0,0.0,1.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,0.0,40000.0,"""CAL""",0.0,0.0,null,0.0,…,null,0.0,0.0,1.0,false,"""OTHER""","""OTHER""",12.0,0.0,0.0,null,"""a55475b1""",null,1.0,1.0,null,0.0,0.0,"""BO""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,10000.0,false
4,"""2019-01-04""",201901,0,1,null,3390.2,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,0.0,44000.0,"""CAL""",0.0,0.0,null,0.0,…,null,0.0,0.0,0.0,false,"""OTHER""","""OTHER""",24.0,0.0,0.0,null,"""a55475b1""",null,0.0,0.0,null,0.0,0.0,"""BO""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,24000.0,false


In [14]:
# new_df = new_df.drop('cntincpaycont9m_3716944L')

In [15]:
df = new_df.drop_nulls()

In [16]:
df.shape

(630292, 123)

In [17]:
y = df['target']
d = df.drop(columns=['target'])

In [18]:
cols = d.columns

In [19]:
lst = []
for col in cols:
    if((d[col]).dtype ==pl.datatypes.Utf8):
        lst.append(col)
lst

['date_decision',
 'credtype_322L',
 'disbursementtype_67L',
 'inittransactioncode_186L',
 'lastactivateddate_801D',
 'lastapplicationdate_877D',
 'lastapprcommoditycat_1041M',
 'lastapprcommoditytypec_5251766M',
 'lastapprdate_640D',
 'lastcancelreason_561M',
 'lastrejectcommoditycat_161M',
 'lastrejectcommodtypec_5251769M',
 'lastrejectreason_759M',
 'lastrejectreasonclient_4145040M',
 'lastst_736L',
 'paytype1st_925L',
 'paytype_783L',
 'previouscontdistrict_112M',
 'twobodfilling_608L',
 'dateofbirth_337D',
 'description_5085714M',
 'education_1103M',
 'education_88M',
 'maritalst_385M',
 'maritalst_893M']

In [20]:
import pandas as pd
df = d.to_pandas()

In [21]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
for col in lst:
    df[col] = label_encoder.fit_transform(df[col])
df

,case_id,date_decision,MONTH,WEEK_NUM,actualdpdtolerance_344P,annuity_780A,annuitynextmonth_57A,applicationcnt_361L,applications30d_658L,applicationscnt_1086L,...,education_88M,firstquarter_103L,fourthquarter_440L,maritalst_385M,maritalst_893M,numberofqueries_373L,secondquarter_766L,thirdquarter_1082L,mainoccupationinc_384A_max,mainoccupationinc_384A_any_selfemployed
0,2297,16,201901,3,0.0,7400.8003,0.0000,0.0,1.0,5.0,...,3,2.0,2.0,2,3,4.0,0.0,1.0,80000.0,False
1,2302,16,201901,3,0.0,3633.6000,4992.8003,0.0,0.0,0.0,...,3,2.0,7.0,2,3,9.0,0.0,0.0,18000.0,False
2,2325,16,201901,3,0.0,4118.4000,2297.4001,0.0,1.0,0.0,...,3,0.0,0.0,2,3,0.0,0.0,0.0,50000.0,False
3,2402,17,201901,3,0.0,4859.0000,0.0000,0.0,0.0,2.0,...,3,2.0,10.0,4,3,10.0,4.0,3.0,60000.0,False
4,2442,17,201901,3,0.0,3551.0000,0.0000,0.0,0.0,0.0,...,3,0.0,0.0,2,3,0.0,0.0,0.0,60000.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
630287,2703443,612,202010,91,0.0,4960.8003,2717.2000,0.0,0.0,0.0,...,3,2.0,5.0,0,3,7.0,2.0,3.0,110000.0,False
630288,2703448,612,202010,91,0.0,8016.0000,0.0000,0.0,0.0,0.0,...,3,8.0,3.0,0,3,5.0,1.0,2.0,100000.0,False
630289,2703450,612,202010,91,0.0,3675.4001,0.0000,0.0,0.0,0.0,...,3,0.0,1.0,2,3,0.0,1.0,1.0,40000.0,False
630290,2703452,612,202010,91,0.0,7788.8003,0.0000,0.0,0.0,0.0,...,3,0.0,1.0,2,3,3.0,0.0,4.0,30000.0,False


In [22]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_data = scaler.fit_transform(df)
pca = PCA(n_components=80)
final_df = pca.fit_transform(scaled_data)
final_df

array([[-1.68634278,  1.3180976 ,  0.25138902, ...,  0.95973489,
         0.08412997, -0.54607893],
       [-1.16081879,  5.03752678, -0.72771812, ...,  0.11403049,
         0.67060018, -0.24667447],
       [-2.64360412, -1.00708986, -0.35560888, ...,  0.83968569,
         0.33975126, -0.41697508],
       ...,
       [ 6.58877855, -5.9552899 ,  6.40590079, ..., -0.1082627 ,
         0.13535472,  0.2710161 ],
       [-1.99780435, -0.64585281,  2.39832455, ...,  0.75088298,
         0.04408897,  0.40226486],
       [ 9.60299614, -4.56286983,  5.39332606, ..., -1.01076854,
        -0.58156077,  0.41887647]])

In [23]:
import numpy as np
components = pca.components_
mif=[]
for i, comp in enumerate(components):
    mifi = np.argmax(np.abs(comp))
    mif.append(cols[mifi])
dropcols = [col for col in cols if col not in mif]
dropcols

['date_decision',
 'annuity_780A',
 'applicationscnt_629L',
 'applicationscnt_867L',
 'clientscnt12m_3712952L',
 'clientscnt3m_3712950L',
 'clientscnt_1022L',
 'clientscnt_1071L',
 'commnoinclast6m_3546845L',
 'credamount_770A',
 'credtype_322L',
 'currdebt_22A',
 'deferredmnthsnum_166L',
 'disbursedcredamount_1113A',
 'disbursementtype_67L',
 'homephncnt_628L',
 'interestrate_311L',
 'isbidproduct_1095L',
 'lastapplicationdate_877D',
 'lastapprdate_640D',
 'lastcancelreason_561M',
 'lastst_736L',
 'mastercontrelectronic_519L',
 'mastercontrexist_109L',
 'maxdpdfrom6mto36m_3546853P',
 'maxdpdlast12m_727P',
 'maxdpdlast6m_474P',
 'maxdpdlast9m_1059P',
 'maxdpdtolerance_374P',
 'mobilephncnt_593L',
 'monthsannuity_845L',
 'numactivecreds_622L',
 'numactivecredschannel_414L',
 'numcontrs3months_479L',
 'numinstlallpaidearly3d_817L',
 'numinstls_657L',
 'numinstlsallpaid_934L',
 'numinstlswithoutdpd_562L',
 'numinstpaidearly3d_3546850L',
 'numinstpaidearly_338L',
 'numinstpaidlate1d_354685

In [24]:
df = df.drop(columns=dropcols)
df

,case_id,MONTH,WEEK_NUM,actualdpdtolerance_344P,annuitynextmonth_57A,applicationcnt_361L,applications30d_658L,applicationscnt_1086L,applicationscnt_464L,clientscnt6m_3712949L,...,education_1103M,education_88M,firstquarter_103L,fourthquarter_440L,maritalst_385M,maritalst_893M,secondquarter_766L,thirdquarter_1082L,mainoccupationinc_384A_max,mainoccupationinc_384A_any_selfemployed
0,2297,201901,3,0.0,0.0000,0.0,1.0,5.0,0.0,0.0,...,3,3,2.0,2.0,2,3,0.0,1.0,80000.0,False
1,2302,201901,3,0.0,4992.8003,0.0,0.0,0.0,2.0,0.0,...,3,3,2.0,7.0,2,3,0.0,0.0,18000.0,False
2,2325,201901,3,0.0,2297.4001,0.0,1.0,0.0,0.0,0.0,...,3,3,0.0,0.0,2,3,0.0,0.0,50000.0,False
3,2402,201901,3,0.0,0.0000,0.0,0.0,2.0,0.0,0.0,...,1,3,2.0,10.0,4,3,4.0,3.0,60000.0,False
4,2442,201901,3,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,...,3,3,0.0,0.0,2,3,0.0,0.0,60000.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
630287,2703443,202010,91,0.0,2717.2000,0.0,0.0,0.0,0.0,0.0,...,2,3,2.0,5.0,0,3,2.0,3.0,110000.0,False
630288,2703448,202010,91,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,...,3,3,8.0,3.0,0,3,1.0,2.0,100000.0,False
630289,2703450,202010,91,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,...,3,3,0.0,1.0,2,3,1.0,1.0,40000.0,False
630290,2703452,202010,91,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,...,3,3,0.0,1.0,2,3,0.0,4.0,30000.0,False


In [25]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier
# Assuming you have a dataframe df with features and a target column 'target'

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.4, random_state=500)

# Initialize and train SVM classifier
clf = RandomForestClassifier(n_estimators=200,n_jobs=-1,verbose=2)
# You can choose different kernels (e.g., 'rbf', 'poly', etc.)
clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


building tree 1 of 200
building tree 2 of 200
building tree 3 of 200
building tree 4 of 200
building tree 5 of 200
building tree 6 of 200
building tree 7 of 200
building tree 8 of 200
building tree 9 of 200
building tree 10 of 200
building tree 11 of 200
building tree 12 of 200
building tree 13 of 200
building tree 14 of 200
building tree 15 of 200
building tree 16 of 200
building tree 17 of 200
building tree 18 of 200
building tree 19 of 200
building tree 20 of 200
building tree 21 of 200
building tree 22 of 200
building tree 23 of 200
building tree 24 of 200
building tree 25 of 200
building tree 26 of 200
building tree 27 of 200
building tree 28 of 200
building tree 29 of 200
building tree 30 of 200
building tree 31 of 200
building tree 32 of 200
building tree 33 of 200
building tree 34 of 200
building tree 35 of 200
building tree 36 of 200
building tree 37 of 200
building tree 38 of 200
building tree 39 of 200


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   22.8s


building tree 40 of 200
building tree 41 of 200
building tree 42 of 200
building tree 43 of 200
building tree 44 of 200
building tree 45 of 200
building tree 46 of 200
building tree 47 of 200
building tree 48 of 200
building tree 49 of 200
building tree 50 of 200
building tree 51 of 200
building tree 52 of 200
building tree 53 of 200
building tree 54 of 200
building tree 55 of 200
building tree 56 of 200
building tree 57 of 200
building tree 58 of 200
building tree 59 of 200
building tree 60 of 200
building tree 61 of 200
building tree 62 of 200
building tree 63 of 200
building tree 64 of 200
building tree 65 of 200
building tree 66 of 200
building tree 67 of 200
building tree 68 of 200
building tree 69 of 200
building tree 70 of 200
building tree 71 of 200
building tree 72 of 200
building tree 73 of 200
building tree 74 of 200
building tree 75 of 200
building tree 76 of 200
building tree 77 of 200
building tree 78 of 200
building tree 79 of 200
building tree 80 of 200
building tree 81

[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  1.7min


building tree 159 of 200
building tree 160 of 200
building tree 161 of 200
building tree 162 of 200
building tree 163 of 200
building tree 164 of 200
building tree 165 of 200
building tree 166 of 200
building tree 167 of 200
building tree 168 of 200
building tree 169 of 200
building tree 170 of 200
building tree 171 of 200
building tree 172 of 200
building tree 173 of 200
building tree 174 of 200
building tree 175 of 200
building tree 176 of 200
building tree 177 of 200
building tree 178 of 200
building tree 179 of 200
building tree 180 of 200
building tree 181 of 200
building tree 182 of 200
building tree 183 of 200
building tree 184 of 200
building tree 185 of 200
building tree 186 of 200
building tree 187 of 200
building tree 188 of 200
building tree 189 of 200
building tree 190 of 200
building tree 191 of 200
building tree 192 of 200
building tree 193 of 200
building tree 194 of 200
building tree 195 of 200
building tree 196 of 200
building tree 197 of 200
building tree 198 of 200


[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    0.8s
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:    3.5s


Accuracy: 0.9671184410412625


[Parallel(n_jobs=4)]: Done 200 out of 200 | elapsed:    4.5s finished
